In [1]:
import unittest

from shapely import LineString, Point
import geopandas as gpd
import geo_nx as gnx 
import networkx as nx 
import pandas as pd
import geo_nx.utils as utils
from networkx.utils import graphs_equal

paris = Point(2.3514, 48.8575)
lyon = Point(4.8357, 45.7640)
marseille = Point(5.3691, 43.3026)
bordeaux = Point(-0.56667, 44.833328)

In [2]:
from itertools import product

def di_neighbors(dgr, node, uturn=False, nopath=None):
    e_nodes = [str(node) + 'e'+ str(pnd) for pnd in dgr.predecessors(node)]
    s_nodes = [str(node) + 's'+ str(snd) for snd in dgr.successors(node)]
    m_edges = list(product(e_nodes, s_nodes))
    if not uturn:
        m_edges = [edge for edge in m_edges if edge[0].replace("e", "s") != edge[1]]
    if nopath:
        no_edges = [ (str(node) + 'e' + str(edge[0]), str(node) + 's' + str(edge[1])) for edge in nopath]
        return list(set(m_edges) - set(no_edges))
    return m_edges

In [3]:
def insert_di_nodes(dgr, node, uturn=False, nopath=None):
    m_edges = di_neighbors(dgr, node, uturn, nopath)
    pred_nodes = list(dgr.predecessors(node))
    succ_nodes = list(dgr.successors(node))
    for pnd in pred_nodes:
        dgr.remove_edge(pnd, node)
        dgr.add_edge(pnd, str(node) + 'e' + str(pnd))
    for snd in succ_nodes:
        dgr.remove_edge(node, snd)
        dgr.add_edge(str(node) + 's' + str(snd), snd)    
    dgr.add_edges_from(m_edges)

In [ ]:
def to_undi_nodes(iter_nodes, path=True):
    for 

In [27]:
"123e456".replace("e", "s")

'123s456'

In [10]:
gr = nx.Graph()
gr.add_edges_from([(1, 2), (1, 3), (3,4), (2,3)])
nx.shortest_path(gr, source=2, target=4)

[2, 3, 4]

In [11]:
dgr = gr.to_directed()
dgr.edges

OutEdgeView([(1, 2), (1, 3), (2, 1), (2, 3), (3, 1), (3, 4), (3, 2), (4, 3)])

In [12]:
nx.shortest_path(dgr, source=2, target=4)

[2, 3, 4]

In [14]:
di_neighbors(dgr, 3)

[('3e1', '3s4'),
 ('3e1', '3s2'),
 ('3e2', '3s1'),
 ('3e2', '3s4'),
 ('3e4', '3s1'),
 ('3e4', '3s2')]

In [15]:
di_neighbors(dgr, 3, nopath=[(2,4)])

[('3e2', '3s1'),
 ('3e4', '3s2'),
 ('3e4', '3s1'),
 ('3e1', '3s2'),
 ('3e1', '3s4')]

In [16]:
dgr = gr.to_directed()
insert_di_nodes(dgr, 3)
nx.shortest_path(dgr, source=2, target=4), nx.shortest_path(dgr, source=4, target=2)

([2, '3e2', '3s4', 4], [4, '3e4', '3s2', 2])

In [17]:
dgr = gr.to_directed()
insert_di_nodes(dgr, 3, nopath=[(2,4)])
nx.shortest_path(dgr, source=2, target=4), nx.shortest_path(dgr, source=4, target=2)

([2, 1, '3e1', '3s4', 4], [4, '3e4', '3s2', 2])